# Skip-gram in Action

## Colab Setup

You can skip this section if not running on Google's colab.

If running with GPUs, sanity check that the GPUs are enabled.

In [2]:
!nvidia-smi

Wed Dec  2 03:20:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
torch.cuda.is_available()

True

The above should be True. If not, debug (Note: version of pytorch I used is not capatible with CUDA drivers on colab. Follow these instructions here explicitly).

In [4]:
!pwd

/content


This should be "/content" on Colab.

First, if running from colab, you must install the package. (You may skip if you installed already).

In [ ]:
!git clone --single-branch --branch colab https://github.com/will-thompson-k/deeplearning-nlp-models.git
%cd deeplearning-nlp-models

In [ ]:
!pip install datasets

In [ ]:
!python setup.py install

## Imports

Here are the packages we need to import.

In [8]:
from nlpmodels.models import word2vec
from nlpmodels.utils import utils, train
from nlpmodels.utils.elt import skipgram_dataset
from argparse import Namespace
import torch
utils.set_seed_everywhere()

## Hyper-parameters

These are the data processing, skip-gram, and model training hyper-parameters for this run.

In [19]:
args = Namespace(
    # skip gram data hyper-parameters
    context_window_size = 5,
    subsample_t = 10.e-5, # param for sub-sampling frequent words (10.e-5 suggested by paper)
    # Model hyper-parameters
    embedding_size = 512,
    negative_sample_size= 20, # k examples to be used in negative sampling loss function
    # Training hyper-parameters
    num_epochs=5,
    learning_rate=0.0001,
    batch_size = 4096,
)

## Get Data

Call the function that grabs training data (via hugging faces) and a dictionary.

In [20]:
train_dataloader, vocab = skipgram_dataset.SkipGramDataset.get_training_dataloader(args.context_window_size,
                                                                                   args.subsample_t,
                                                                                   args.batch_size)

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


In [21]:
vocab_size = len(vocab)

print(f"The gist: context_window_size = {args.context_window_size}, "
      f"batch_size = {args.batch_size}, vocab_size = {vocab_size}, "
      f"embedding_size = {args.embedding_size}, k = {args.negative_sample_size}, "
      f"train_size = {len(train_dataloader.dataset)}"
      )

The gist: context_window_size = 5, batch_size = 4096, vocab_size = 61811, embedding_size = 512, k = 20, train_size = 16103772


## Training

Here we build the model and call the trainer.

In [22]:
word_frequencies = torch.from_numpy(vocab.get_word_frequencies())
model = word2vec.SkipGramNSModel(vocab_size, args.embedding_size, args.negative_sample_size,word_frequencies)
trainer = train.Word2VecTrainer(args,model,train_dataloader)
trainer.run()

[Epoch 4]: 100%|██████████| 3932/3932 [04:05<00:00, 16.00it/s, loss=0.703]

Finished Training...


## Examine Similarity of Embeddings

Now that we've trained our embeddings, let's see if the words that are clustered together make any sense.

We will use cosine similarity to find the embeddings that are most similar in the embeddings space. This is one metric
for similarity. Another popular metric is based on euclidean distance. To use that metric, check out pytorch's
cdist() function. Also, can't speak highly enough of `spotify::annoy` package.

In [23]:
embeddings = model.get_embeddings().to(torch.device('cpu'))

In [24]:
embeddings

tensor([[-6.1830e-04, -6.3485e-04,  6.9642e-04,  ..., -3.4787e-04,
         -8.6980e-04,  2.1303e-04],
        [-9.1912e-04, -7.5858e-04, -9.1346e-04,  ..., -6.2681e-04,
          4.7648e-04,  5.9994e-04],
        [-9.0853e-04,  6.7023e-04, -2.8429e-05,  ...,  4.2949e-04,
          6.4231e-04, -2.3924e-04],
        ...,
        [ 1.4746e-02, -1.4129e-02,  1.6054e-02,  ...,  1.4172e-02,
         -1.5293e-02,  1.5223e-02],
        [ 4.1555e-02, -4.0126e-02,  4.1399e-02,  ...,  4.1131e-02,
         -4.0341e-02,  4.0407e-02],
        [ 3.4955e-02, -3.6053e-02,  3.5510e-02,  ...,  3.4885e-02,
         -3.5795e-02,  3.4757e-02]])

### Computer

Let's see the top 10 words associated with "computer".

In [25]:
utils.get_cosine_similar("computer",vocab._token_to_idx,embeddings)[0:10]

[('financial', tensor(0.9999)),
 ('force', tensor(0.9999)),
 ('low', tensor(0.9999)),
 ('india', tensor(0.9999)),
 ('stock', tensor(0.9999)),
 ('called', tensor(0.9999)),
 ('action', tensor(0.9999)),
 ('saying', tensor(0.9999)),
 ('official', tensor(0.9999)),
 ('so', tensor(0.9999))]

### Market

Let's see the top 5 words associated with "market".

In [26]:
utils.get_cosine_similar("market",vocab._token_to_idx,embeddings)[0:10]

[('but', tensor(0.9999)),
 ('with', tensor(0.9999)),
 ('as', tensor(0.9999)),
 ('for', tensor(0.9999)),
 ('after', tensor(0.9999)),
 ('final', tensor(0.9999)),
 ('an', tensor(0.9999)),
 ('against', tensor(0.9999)),
 ('over', tensor(0.9999)),
 ('us', tensor(0.9999))]